In [1]:
import pandas as pd
import yaml

yaml_path = "configs/base.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)


data_NQ = pd.read_csv(config["data"]["raw_NQ"])  

In [ ]:
# ...

NQ_RSI = rsi(data_NQ, period=2)
NQ_Hurst = hurst_exponent(data_NQ, window=20)
NQ_composite_RSI = composite_RSI(period=24)

In [ ]:
import matplotlib.pyplot as plt


fig, ax_price = plt.subplots(figsize=(12, 6))

# PRICE
ax_price.plot(data_NQ["Close"], label="NQ Close", color="black")
ax_price.set_ylabel("Prezzo")
ax_price.legend(loc="upper left")

# RSI 
ax_rsi = ax_price.twinx()
ax_rsi.plot(NQ_RSI, label="RSI", alpha=0.6)
ax_rsi.set_ylabel("RSI")
ax_rsi.spines["right"].set_position(("axes", 1.05))  
ax_rsi.legend(loc="upper right")

# HURST 
ax_hurst = ax_price.twinx()
ax_hurst.plot(NQ_Hurst, label="Hurst", alpha=0.6)
ax_hurst.set_ylabel("Hurst")
ax_hurst.spines["right"].set_position(("axes", 1.10))
ax_hurst.legend(loc="lower right")

# COMPOSITE RSI 
ax_comp = ax_price.twinx()
ax_comp.plot(NQ_composite_RSI, label="Composite RSI", alpha=0.6)
ax_comp.set_ylabel("Composite RSI")
ax_comp.spines["right"].set_position(("axes", 1.15))
ax_comp.legend(loc="center right")

plt.title("NQ Price with RSI, Hurst, and Composite RSI")
plt.tight_layout()
plt.show()